In [ ]:
TSCALE = 4
import numpy
import pandas
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from mpl_toolkits.basemap import Basemap, cm
import cartopy.crs as ccrs
from datetime import datetime, timedelta, date
import model
import modelsim
import ftemp
import vtemp
import culex
import priorQ
import priorP

# Figure 1

In [ ]:
paramQ = modelsim.paramQ.copy()
paramP = modelsim.paramP.copy()

model.plotPDC(paramQ, ylog=True, subset=True, filename="figuresLT/param_Cxquin_C", filetype='pdf')
model.plotPDC(paramP, ylog=True, filename="figuresLT/param_Cxpip_C", filetype='pdf')

# Figure 2

In [ ]:
dd = {}
for i in vtemp.obs:
    o = vtemp.obs[i]
    if numpy.any(o['A'][1:]<o['A'][:-1]):
        print("Error:",o['A'])
        break
    d = {
        'Date': o['Date'][0]
    }
    if numpy.all(o['A']==0):
        d['A'] = numpy.nan
        d['lA'] = numpy.nan
    else:
        j = numpy.where(o['A']>o['A'][-1]*0.5)[0][0]
        d['A'] = 100.0*o['A'][-1]/o['E'][0]
        d['lA'] = (o['Date'][j]-o['Date'][0]).days
    dd[i] = d

rets = numpy.load("mat/data_era5_paramP_2017_2018.npy",allow_pickle=True)
ret = rets[:,:,1:]
tm, pp, m = modelsim.calcRet(ret)

plt.rcParams.update({'font.size': 14})
plt.rcParams['figure.figsize'] = [12, 4]
fig, ax = plt.subplots()
ax.fill_between(tm,pp[0][:,0],pp[2][:,0],alpha=0.35,color="blue")
ax.plot(tm,pp[1][:,0],alpha=1,lw=3,color="blue",label="No. of adults")
ax.set_ylabel("No. of adults",fontsize=14,labelpad=0)

ax1 = ax.twinx()  
ax1.fill_between(tm,pp[0][:,1],pp[2][:,1],alpha=0.35,color="green")
ax1.plot(tm,pp[1][:,1],alpha=1,lw=3,color="green",label="Days to emergence")
ax1.set_ylim(0,150)
ax1.set_ylabel("Days to emergence",fontsize=14,labelpad=0)

ax.xaxis.set_major_formatter(mdates.DateFormatter('%d %b %Y'))
ax.xaxis.set_major_locator(plt.MaxNLocator(7))
ax.tick_params(axis='x', which='major', rotation=15)
ax.set_xlim(date(2017,1,1),date(2018,1,31))
ax.set_ylim(0,100)
#
for i in dd:
    d = dd[i]
    ax.plot(d['Date'],d['A'],'o',ms=9,color="black")
    ax1.plot(d['Date'],d['lA'],'o',ms=9,color="black")
    ax.plot(d['Date'],d['A'],'o',ms=7,color="blue")
    ax1.plot(d['Date'],d['lA'],'o',ms=7,color="green")
    if i in [12,10]:
        ax.plot(d['Date'],d['A'],'o',ms=16,markeredgewidth=2,markerfacecolor='none',markeredgecolor="black")
        ax1.plot(d['Date'],d['lA'],'o',ms=16,markeredgewidth=2,markerfacecolor='none',markeredgecolor="black",label="Training data" if i==10 else None)
#
lgnd = ax.legend(loc='upper left')
for l in lgnd.legendHandles:
    l.set_linewidth(4)
    l.set_alpha(1)
lgnd = ax1.legend(loc='upper right')
for l in lgnd.legendHandles:
    l.set_linewidth(4)
    l.set_alpha(1)
#
left, bottom, width, height = [0.66, 0.25, 0.22, 0.27]
ax2 = fig.add_axes([left, bottom, width, height])
h = ax2.hist(m,bins=20,color="blue",alpha=0.65)
ax2.patch.set_alpha(0.0)
ax2.text(numpy.min(m),0.35*numpy.max(h[0]),"First emergence",fontsize=14)
ax2.yaxis.set_major_locator(plt.NullLocator())
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%b%d'))
ax2.xaxis.set_major_locator(plt.MaxNLocator(5))
ax2.tick_params(axis='both', which='major', labelsize=12, rotation=15)
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
ax2.spines['left'].set_visible(False)
#
plt.savefig("figuresLT/global_TTE_ERA5_P_data_V2.pdf",bbox_inches="tight",dpi=300)
plt.show()

# Figure 3

In [ ]:
nasa_models = ['ACCESS1-0', 'CCSM4', 'CNRM-CM5']
years = [2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026]
rets = {}
for m in nasa_models:
    for y in years:
        filename = "mat/data_nasa_paramP_lon19.852lat45.246_F50_R3_%s_%d_%d.npy" %(m, y, y+50)
        rets[m+str(y)] = numpy.load(filename,allow_pickle=True)

ret0 = numpy.vstack([numpy.stack([rets[m+str(y)][n,rets[m+str(y)][n,:,0]==0,1:] for n in range(rets[m+str(y)].shape[0])]) for y in years for m in nasa_models])
ret1 = numpy.vstack([numpy.stack([rets[m+str(y)][n,rets[m+str(y)][n,:,0]==1,1:] for n in range(rets[m+str(y)].shape[0])]) for y in years for m in nasa_models])
ret01 = numpy.vstack([numpy.stack([rets[m+str(y)][n,rets[m+str(y)][n,:,0]==1,1:]-rets[m+str(y)][n,rets[m+str(y)][n,:,0]==0,1:] for n in range(rets[m+str(y)].shape[0])]) for y in years for m in nasa_models])
ret01[0,:,0] = numpy.vstack([numpy.stack([rets[m+str(y)][n,rets[m+str(y)][n,:,0]==0,1] for n in range(rets[m+str(y)].shape[0])]) for y in years for m in nasa_models])[0,:]
tm0, pp0, m0 = modelsim.calcRet(ret0)
tm1, pp1, m1 = modelsim.calcRet(ret1)
tm01, pp01, m01 = modelsim.calcRet(ret01)

plt.rcParams.update({'font.size': 14})
plt.rcParams['figure.figsize'] = [12, 4]
fig, ax = plt.subplots()
ax.fill_between(tm0,pp0[0][:,0],pp0[2][:,0],alpha=0.35,color="blue")
ax.plot(tm0,pp0[1][:,0],alpha=1,lw=3,color="blue",label="No. of adults (2017-2027)")
ax.fill_between(tm0,pp1[0][:,0],pp1[2][:,0],alpha=0.35,color="navy")
ax.plot(tm0,pp1[1][:,0],alpha=1,lw=3,color="navy",label="No. of adults (2067-2077)")
ax.set_ylabel("No. of adults",fontsize=14,labelpad=0)

ax1 = ax.twinx()  
ax1.fill_between(tm0,pp0[0][:,1],pp0[2][:,1],alpha=0.35,color="green")
ax1.plot(tm0,pp0[1][:,1],alpha=1,lw=3,color="green",label="Days to emergence (2017-2027)")
ax1.fill_between(tm0,pp1[0][:,1],pp1[2][:,1],alpha=0.35,color="darkolivegreen")
ax1.plot(tm0,pp1[1][:,1],alpha=1,lw=3,color="darkolivegreen",label="Days to emergence (2067-2077)")
ax1.set_ylim(0,150)
ax1.set_ylabel("Days to emergence",fontsize=14,labelpad=0)

ax.xaxis.set_major_formatter(mdates.DateFormatter('%d %b %Y'))
ax.xaxis.set_major_locator(plt.MaxNLocator(7))
ax.tick_params(axis='x', which='major', rotation=15)
ax.set_xlim(date(2017,1,1),date(2018,1,31))
ax.set_ylim(0,100)
#
lgnd = ax.legend(loc='upper left')
for l in lgnd.legendHandles:
    l.set_linewidth(4)
    l.set_alpha(1)
lgnd = ax1.legend(loc='upper right')
for l in lgnd.legendHandles:
    l.set_linewidth(4)
    l.set_alpha(1)
#
left, bottom, width, height = [0.67, 0.25, 0.22, 0.27]
ax2 = fig.add_axes([left, bottom, width, height])
mm1 = [(i-timedelta(days=(date(2067,1,1)-date(2017,1,1)).days)) for i in m1]
h = ax2.hist(m0,bins=20,color="blue",alpha=0.65)
h = ax2.hist(mm1,bins=20,color="navy",alpha=0.65)
ax2.patch.set_alpha(0.0)
ax2.text(numpy.max(m0)-timedelta(days=0),1.25*numpy.max(h[0]),"First\nemergence",fontsize=14,horizontalalignment='right')
ax2.yaxis.set_major_locator(plt.NullLocator())
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%b%d'))
ax2.xaxis.set_major_locator(plt.MaxNLocator(5))
ax2.tick_params(axis='both', which='major', labelsize=12, rotation=15)
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
ax2.spines['left'].set_visible(False)
plt.savefig("figuresLT/global_TTE_ERA5_NASA_P_V2.pdf",bbox_inches="tight",dpi=300)
plt.show()

# Figure 4

In [ ]:
srt = pandas.read_csv("mat/global_ERA5_Cxpip_y2x3_srt.csv")
prd = pandas.read_csv("mat/global_ERA5_Cxpip_y2x3_prd.csv")

def plotERA5():
    mat = numpy.ndarray((721,1440),dtype=numpy.float64)
    mat[:] = numpy.nan
    mat[numpy.int32((y+90)*4),numpy.int32((x+180)*4-1)] = z
    mat = mat[121:,:]
    #
    cmap = plt.cm.get_cmap('viridis')
    if cmaprev:
        cmap = cmap.reversed()
    norm = mpl.colors.BoundaryNorm(bins, cmap.N, clip=True)
    plt.rcParams.update({'font.size': 14})
    plt.rcParams['figure.figsize'] = [12, 4]
    m = Basemap(projection='cyl',llcrnrlat=-60,urcrnrlat=90,llcrnrlon=-180,urcrnrlon=180,resolution='c')
    m.drawcoastlines(linewidth=1.0,zorder=-1)
    img = m.imshow(mat,cmap=cmap,norm=norm,interpolation='none')
    cb = m.colorbar(size=0.1, pad=0.05, boundaries=bins, extend=extend)
    if len(ticks) and len(ticklabs):
        cb.set_ticks(ticks)
        cb.set_ticklabels(ticklabs)
    plt.savefig(filename,bbox_inches="tight",dpi=300)
    plt.show()

x = prd['lon']
y = prd['lat']
z = prd['mn'].copy()
bins = numpy.linspace(0,100,6)
ticks = []
ticklabs = []
extend = 'neither'
cmaprev = False
filename = "figuresLT/global_ERA5_map_Cxpip_PRODA_mn.pdf"
plotERA5()

x = prd['lon']
y = prd['lat']
z = prd['mnstd'].copy()
bins = numpy.linspace(0,40,5)
ticks = []
ticklabs = []
extend = 'max'
cmaprev = False
filename = "figuresLT/global_ERA5_map_Cxpip_PRODA_mnstd.pdf"
plotERA5()

x = srt['lon']
y = srt['lat']
z = srt['mn'].copy()
bins = numpy.arange(0,90,15)
ticks = []
ticklabs = []
extend = 'max'
cmaprev = False
filename = "figuresLT/global_ERA5_map_Cxpip_DEVTM_mn.pdf"
plotERA5()

x = srt['lon']
y = srt['lat']
z = srt['mnstd'].copy()
bins = numpy.arange(0,14,3)
ticks = []
ticklabs = []
extend = 'max'
cmaprev = False
filename = "figuresLT/global_ERA5_map_Cxpip_DEVTM_mnstd.pdf"
plotERA5()

x = srt['lon']
y = srt['lat']
z = srt['first'].copy()
bins = numpy.array([1,3,5,7,9,11,13])
ticks = bins[:-1]+1
ticklabs = ['Feb\nJan','Apr\nMar','Jun\nMay','Aug\nJul','Oct\nSep','Dec\nNov']
extend = 'neither'
cmaprev = True
filename = "figuresLT/global_ERA5_map_Cxpip_DEVTM_first.pdf"
plotERA5()

# Figure 5

In [ ]:
srt01 = pandas.read_csv("mat/global_NASA_Cxpip_R3_srt01.csv")
prd01 = pandas.read_csv("mat/global_NASA_Cxpip_R3_prd01.csv")

def plotNASA():
    mat = numpy.ndarray((360,720),dtype=numpy.float64)
    mat[:] = numpy.nan
    mat[numpy.int32(((y+90)*4)/2),numpy.int32(((x+180)*4-1)/2)] = z
    mat = mat[61:,:]
    #
    if nrm2s:
        cmap = plt.cm.get_cmap('RdBu_r')
        if cmaprev:
            cmap = cmap.reversed()
        norm = mpl.colors.TwoSlopeNorm(0,nrm2s[0],nrm2s[1])
    else:
        cmap = plt.cm.get_cmap('viridis')
        if cmaprev:
            cmap = cmap.reversed()
        norm = mpl.colors.BoundaryNorm(bins, cmap.N, clip=True)
    #
    plt.rcParams.update({'font.size': 14})
    plt.rcParams['figure.figsize'] = [12, 4]
    m = Basemap(projection='cyl',llcrnrlat=-60,urcrnrlat=90,llcrnrlon=-180,urcrnrlon=180,resolution='c')
    m.drawcoastlines(linewidth=1.0,zorder=-1)
    img = m.imshow(mat,cmap=cmap,norm=norm,interpolation='none')
    cb = m.colorbar(size=0.1, pad=0.05, boundaries=bins, extend=extend)
    if len(ticks) and len(ticklabs):
        cb.set_ticks(ticks)
        cb.set_ticklabels(ticklabs)
    plt.savefig(filename,bbox_inches="tight",dpi=300)
    plt.show()

x = prd01['lon']
y = prd01['lat']
z = prd01['mn'].copy()
bins = numpy.arange(-30,41,10)
nrm2s = [-30,40]
ticks = []
ticklabs = []
extend = 'both'
cmaprev = False
filename = "figuresLT/global_NASA_2017_map_Cxpip_PRODA_01_mn.pdf"
plotNASA()

x = prd01['lon']
y = prd01['lat']
z = prd01['mnstd'].copy()
bins = numpy.linspace(0,60,7)
nrm2s = []
ticks = []
ticklabs = []
extend = 'max'
cmaprev = False
filename = "figuresLT/global_NASA_2017_map_Cxpip_PRODA_01_mnstd.pdf"
plotNASA()

x = srt01['lon']
y = srt01['lat']
z = srt01['mn'].copy()
bins = numpy.arange(-21,22,7)
nrm2s = [-14,14]
ticks = []
ticklabs = []
extend = 'both'
cmaprev = False
filename = "figuresLT/global_NASA_2017_map_Cxpip_DEVTM_01_mn.pdf"
plotNASA()

x = srt01['lon']
y = srt01['lat']
z = srt01['mnstd'].copy()
bins = numpy.arange(0,29,7)
nrm2s = []
ticks = []
ticklabs = []
extend = 'max'
cmaprev = False
filename = "figuresLT/global_NASA_2017_map_Cxpip_DEVTM_01_mnstd.pdf"
plotNASA()

x = srt01['lon']
y = srt01['lat']
z = srt01['firstday'].copy()
bins = numpy.arange(-40,21,10)
nrm2s = [-40, 20]
ticks = []
ticklabs = []
extend = 'both'
cmaprev = True
filename = "figuresLT/global_NASA_2017_map_Cxpip_DEVTM_01_first.pdf"
plotNASA()